In [2]:
from gym_trading_env.downloader import download
from typing import Dict
import datetime
import pandas as pd
import glob
import numpy as np
import gymnasium as gym
import gym_trading_env
import torch
from environments.environments import NeuralForecastingTradingEnv
from neuralforecast import NeuralForecast
from utils.utils import sharpe_reward,build_market_image,prep_forecasts
from utils.mappings import symbol_map
import tqdm.autonotebook as tqdm

In [3]:
torch.mps.is_available()

True

In [4]:
# class MultiPorfolio():
torch.tensor(0).device

device(type='cpu')

In [13]:
agent=torch.load(map_location=torch.device('cpu'),f='Agent/pearl_DOGEUSDTSPOT_model.pkl',)
# torch.load()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [4]:
class MultiAssetTradingEnv():

    def __init__(self,assets,time_frame,model,positions) -> None:
        self.assets=assets
        self.time_frame=time_frame
        self.model=model
        self.positions=positions
        self.action_map=self.make_action_map()
        self.env_map=self.make_env_map()
    
    def build_env_config(self,pair):

        symbol =  pair.replace('/','')
        context_length=max([sub_model.hparams['input_size'] for sub_model in self.model.models])


        data=build_market_image(target_pair=pair,time_frame=time_frame,axis=0)
        data=data[data['symbol']==symbol].copy()

        forecasts,train_df=prep_forecasts(data,model=model)
        env_config=dict(
                        name=f'{symbol}_train',
                        forecast_horizon=model.h,
                        context_length=context_length,
                        initial_position=0,
                        reward_function=sharpe_reward,
                        model=model,
                        df = train_df, # Your dataset with your custom features
                        positions = self.positions, # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                        max_episode_duration=168,
                        verbose=0
                        )
        
        return env_config
    

    def get_action_for_env(self,action):
        action=self.action_map[action]
        for symb,act_list in self.env_map.items():
            if action in act_list:
                env=self.env_map[symb]
                break

        return env,action
        

    def make_action_map(self):
        action_map={}
        action_id=0
        for symbol in self.assets:
            env_action_id=0
            for p in positions:
                action_map[action_id]=env_action_id
                env_action_id+=1
                action_id+=1

        return action_map
    
    
    def make_env_map(self)-> Dict:
        env_map={}
        for symbol in self.assets:
            config= self.build_env_config(symbol)
            env=NeuralForecastingTradingEnv(config)
            env_map[symbol]=env
        
        return env_map


In [5]:
self=MultiAssetTradingEnv(
    assets=COIN_PAIRS,
    time_frame=time_frame,
    model=model,
    positions=positions

    )

In [12]:
# self.build_config(COIN_PAIRS[0])

In [7]:
base_envs={}
symbol='ETHUSD'
for pair in COIN_PAIRS:
    env_config=self.build_config(pair=pair,)
    train_env=NeuralForecastingTradingEnv(env_config)
    base_envs[symbol]=train_env


base_envs

UserWarning: Predict insample might not provide accurate predictions for                        recurrent model AutoLSTM class yet due to scaling.

In [10]:
env_map={}
action_id=0
for symbol in COIN_PAIRS:
    env_map[symbol]=[]
    env_action_id=0

    for p in positions:
        env_map[symbol].append(action_id)
        env_action_id+=1
        action_id+=1

env_map

{'BTC/USDT': [0, 1, 2, 3, 4],
 'ETH/USDT': [5, 6, 7, 8, 9],
 'SOL/USDT': [10, 11, 12, 13, 14],
 'BNB/USDT': [15, 16, 17, 18, 19],
 'XRP/USDT': [20, 21, 22, 23, 24],
 'ADA/USDT': [25, 26, 27, 28, 29]}

In [8]:
action_map={}
action_id=0
for symbol in COIN_PAIRS:
    env_action_id=0
    for p in positions:
        action_map[action_id]=env_action_id
        env_action_id+=1
        action_id+=1

action_map

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 0,
 6: 1,
 7: 2,
 8: 3,
 9: 4,
 10: 0,
 11: 1,
 12: 2,
 13: 3,
 14: 4,
 15: 0,
 16: 1,
 17: 2,
 18: 3,
 19: 4,
 20: 0,
 21: 1,
 22: 2,
 23: 3,
 24: 4,
 25: 0,
 26: 1,
 27: 2,
 28: 3,
 29: 4}